# Wildfire Prediction Model - Test Cases
## Comprehensive Testing Suite for MVP Demo

This notebook contains multiple test cases to validate the pipeline.

In [3]:
# TEST SETUP: Import libraries
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_score, classification_report
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set paths
PROJECT_ROOT = r"G:\UAAS\Wildlife-prediction-try02"
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "raw")
METADATA_CSV = os.path.join(PROJECT_ROOT, "data", "patch_metadata.csv")
SENTINEL_TIF = os.path.join(DATA_DIR, "s2_2021_06_input_10m.tif")
PATCH_SIZE = 64
BATCH_SIZE = 32

print("✓ Libraries imported successfully")
print(f"✓ Project root: {PROJECT_ROOT}")

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


✓ Libraries imported successfully
✓ Project root: G:\UAAS\Wildlife-prediction-try02


In [3]:
!pip install tensorflow 
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 50.5 MB/s eta 0:00:00:00:0100:01


---
## TEST 1: Data Integrity Check
Validates that data files exist and are accessible

In [5]:
def test_data_files_exist():
    """TEST 1: Verify all required data files exist"""
    print("\n" + "="*60)
    print("TEST 1: Data Files Integrity")
    print("="*60)
    
    files_to_check = [
        (METADATA_CSV, "Patch Metadata CSV"),
        (SENTINEL_TIF, "Sentinel-2 GeoTIFF")
    ]
    
    all_exist = True
    for filepath, name in files_to_check:
        exists = os.path.exists(filepath)
        status = "✓ EXISTS" if exists else "✗ MISSING"
        print(f"{status}: {name}")
        print(f"  Path: {filepath}")
        all_exist = all_exist and exists
    
    # Check file sizes
    if os.path.exists(METADATA_CSV):
        size_mb = os.path.getsize(METADATA_CSV) / (1024*1024)
        print(f"  Size: {size_mb:.2f} MB")
    
    if os.path.exists(SENTINEL_TIF):
        size_mb = os.path.getsize(SENTINEL_TIF) / (1024*1024)
        print(f"  Size: {size_mb:.2f} MB")
    
    return all_exist

result_1 = test_data_files_exist()
print(f"\n{'PASSED ✓' if result_1 else 'FAILED ✗'}")


TEST 1: Data Files Integrity
✗ MISSING: Patch Metadata CSV
  Path: G:\UAAS\Wildlife-prediction-try02/data/patch_metadata.csv
✗ MISSING: Sentinel-2 GeoTIFF
  Path: G:\UAAS\Wildlife-prediction-try02/data/raw/s2_2021_06_input_10m.tif

FAILED ✗


---
## TEST 2: CSV Metadata Validation
Checks CSV structure and label distribution

In [ ]:
def test_csv_metadata():
    """TEST 2: Validate CSV metadata structure and content"""
    print("\n" + "="*60)
    print("TEST 2: CSV Metadata Validation")
    print("="*60)
    
    try:
        df = pd.read_csv(METADATA_CSV)
        print(f"✓ CSV loaded successfully")
        print(f"  Shape: {df.shape} (rows x columns)")
        print(f"  Columns: {list(df.columns)}")
        
        # Check for required columns
        required_cols = ['burn_label']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if not missing_cols:
            print(f"✓ All required columns present")
        else:
            print(f"✗ Missing columns: {missing_cols}")
            return False
        
        # Check for missing values
        null_counts = df.isnull().sum()
        if null_counts.sum() == 0:
            print(f"✓ No null values found")
        else:
            print(f"⚠ Null values detected:")
            print(null_counts[null_counts > 0])
        
        # Label distribution
        print(f"\n--- Label Distribution ---")
        label_dist = df['burn_label'].value_counts().sort_index()
        for label, count in label_dist.items():
            pct = (count / len(df)) * 100
            print(f"  Class {label}: {count:6d} ({pct:5.2f}%)")
        
        # Class imbalance ratio
        if len(label_dist) == 2:
            ratio = label_dist[1] / label_dist[0]
            print(f"\n  Imbalance Ratio (Burn/No-Burn): {ratio:.4f}")
        
        return True
    
    except Exception as e:
        print(f"✗ Error reading CSV: {e}")
        return False

result_2 = test_csv_metadata()
print(f"\n{'PASSED ✓' if result_2 else 'FAILED ✗'}")

---
## TEST 3: GeoTIFF Image Validation
Checks raster dimensions, bands, and data types

In [ ]:
def test_geotiff_integrity():
    """TEST 3: Validate GeoTIFF structure and properties"""
    print("\n" + "="*60)
    print("TEST 3: GeoTIFF Integrity Check")
    print("="*60)
    
    try:
        with rasterio.open(SENTINEL_TIF) as src:
            print(f"✓ GeoTIFF opened successfully")
            
            # Basic properties
            print(f"\n--- Raster Properties ---")
            print(f"  Dimensions (W x H): {src.width} x {src.height}")
            print(f"  Number of bands: {src.count}")
            print(f"  Data type: {src.dtypes[0]}")
            print(f"  CRS: {src.crs}")
            
            # Expected for Sentinel-2 10m data
            expected_bands = 3  # B2, B3, B4 (10m resolution)
            if src.count == expected_bands:
                print(f"✓ Expected {expected_bands} bands (Sentinel-2)")
            else:
                print(f"⚠ Expected {expected_bands} bands, got {src.count}")
            
            # Read sample data
            print(f"\n--- Data Statistics ---")
            for band in range(1, min(src.count+1, 4)):
                data = src.read(band)
                print(f"  Band {band}:")
                print(f"    Min: {data.min()}, Max: {data.max()}, Mean: {data.mean():.2f}")
                print(f"    Nodata count: {np.sum(data == src.nodata)}")
            
            return True
    
    except Exception as e:
        print(f"✗ Error reading GeoTIFF: {e}")
        return False

result_3 = test_geotiff_integrity()
print(f"\n{'PASSED ✓' if result_3 else 'FAILED ✗'}")

---
## TEST 4: Patch Extraction & Normalization
Tests the core data loading pipeline

In [ ]:
def load_all_patches_into_memory(metadata_path, sentinel_path, patch_size=64):
    """Load patches from GeoTIFF - core data loading function"""
    print("Loading large GeoTIFF into memory and extracting patches...")
    
    df = pd.read_csv(metadata_path)
    
    try:
        with rasterio.open(sentinel_path) as src:
            full_image_array = src.read()
            img_width, img_height = src.width, src.height
    except Exception as e:
        print(f"Error loading TIF: {e}")
        return None, None, None
    
    full_image_array = np.transpose(full_image_array, (1, 2, 0))
    
    patch_list = []
    label_list = []
    patch_count = 0
    nan_count = 0
    
    for y in range(0, img_height, patch_size):
        for x in range(0, img_width, patch_size):
            if x + patch_size > img_width or y + patch_size > img_height:
                continue 
            
            patch = full_image_array[y:y + patch_size, x:x + patch_size, :]
            patch = patch / 10000.0  # Normalize Sentinel-2
            
            if np.isnan(patch).any():
                nan_count += 1
                patch_count += 1
                continue
            
            patch_list.append(patch)
            if patch_count < len(df):
                label_list.append(df.iloc[patch_count]['burn_label'])
            patch_count += 1
    
    X = np.stack(patch_list).astype(np.float32)
    y = np.array(label_list)
    
    return X, y, df


def test_patch_extraction():
    """TEST 4: Test patch extraction and normalization"""
    print("\n" + "="*60)
    print("TEST 4: Patch Extraction & Normalization")
    print("="*60)
    
    try:
        X, y, df = load_all_patches_into_memory(METADATA_CSV, SENTINEL_TIF, PATCH_SIZE)
        
        if X is None:
            print("✗ Failed to load patches")
            return False
        
        print(f"✓ Patches extracted successfully")
        print(f"\n--- Dataset Shape ---")
        print(f"  X shape: {X.shape} (samples, height, width, channels)")
        print(f"  y shape: {y.shape}")
        print(f"  Total patches: {len(X)}")
        
        # Data validation
        print(f"\n--- Data Quality ---")
        print(f"  X dtype: {X.dtype}")
        print(f"  X min: {X.min():.6f}, max: {X.max():.6f}, mean: {X.mean():.6f}")
        print(f"  NaN values: {np.isnan(X).sum()}")
        print(f"  Inf values: {np.isinf(X).sum()}")
        
        # Label check
        print(f"\n--- Label Distribution ---")
        unique, counts = np.unique(y, return_counts=True)
        for label, count in zip(unique, counts):
            pct = (count / len(y)) * 100
            print(f"  Class {label}: {count:6d} ({pct:5.2f}%)")
        
        # Normalization check (should be between 0-1)
        if X.max() <= 1.0 and X.min() >= 0.0:
            print(f"\n✓ Data properly normalized (0-1 range)")
        else:
            print(f"⚠ Data may not be properly normalized")
        
        return True, X, y, df
    
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False, None, None, None

result_4, X, y, df = test_patch_extraction()
print(f"\n{'PASSED ✓' if result_4 else 'FAILED ✗'}")

---
## TEST 5: Train-Test Split
Validates stratified splitting preserves class distribution

In [ ]:
def test_train_test_split():
    """TEST 5: Validate train-test split stratification"""
    print("\n" + "="*60)
    print("TEST 5: Train-Test Split Validation")
    print("="*60)
    
    if X is None or y is None:
        print("✗ No data loaded")
        return False
    
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.25, random_state=42, stratify=y
        )
        
        print(f"✓ Stratified split successful")
        print(f"\n--- Split Sizes ---")
        print(f"  Training set: {len(X_train)} samples")
        print(f"  Test set: {len(X_test)} samples")
        print(f"  Total: {len(X_train) + len(X_test)} samples")
        
        # Check class distribution
        print(f"\n--- Class Distribution ---")
        print(f"\n  Original:")
        orig_unique, orig_counts = np.unique(y, return_counts=True)
        for label, count in zip(orig_unique, orig_counts):
            pct = (count / len(y)) * 100
            print(f"    Class {label}: {count:6d} ({pct:5.2f}%)")
        
        print(f"\n  Training:")
        train_unique, train_counts = np.unique(y_train, return_counts=True)
        for label, count in zip(train_unique, train_counts):
            pct = (count / len(y_train)) * 100
            print(f"    Class {label}: {count:6d} ({pct:5.2f}%)")
        
        print(f"\n  Test:")
        test_unique, test_counts = np.unique(y_test, return_counts=True)
        for label, count in zip(test_unique, test_counts):
            pct = (count / len(y_test)) * 100
            print(f"    Class {label}: {count:6d} ({pct:5.2f}%)")
        
        return True, X_train, X_test, y_train, y_test
    
    except Exception as e:
        print(f"✗ Error: {e}")
        return False, None, None, None, None

result_5, X_train, X_test, y_train, y_test = test_train_test_split()
print(f"\n{'PASSED ✓' if result_5 else 'FAILED ✗'}")

---
## TEST 6: Model Architecture
Tests that model can be created and compiled

In [ ]:
def create_enhanced_cnn(input_shape, num_classes=2):
    """Creates enhanced CNN with residual blocks"""
    
    def res_block(x, filters, kernel_size=3):
        y = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
        y = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')(y)
        x = tf.keras.layers.Conv2D(filters, 1)(x) 
        z = tf.keras.layers.Add()([x, y])
        return tf.keras.layers.ReLU()(z)

    inputs = tf.keras.layers.Input(shape=input_shape)
    
    x = tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    x = res_block(x, 32)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    x = res_block(x, 64)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model


def test_model_creation():
    """TEST 6: Test model creation and compilation"""
    print("\n" + "="*60)
    print("TEST 6: Model Architecture Validation")
    print("="*60)
    
    try:
        model = create_enhanced_cnn(input_shape=(PATCH_SIZE, PATCH_SIZE, 3))
        print(f"✓ Model created successfully")
        
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        print(f"✓ Model compiled successfully")
        
        # Summary
        print(f"\n--- Model Summary ---")
        model.summary()
        
        # Count parameters
        total_params = model.count_params()
        trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
        print(f"\n--- Parameters ---")
        print(f"  Total parameters: {total_params:,}")
        print(f"  Trainable parameters: {trainable_params:,}")
        
        return True, model
    
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False, None

result_6, model = test_model_creation()
print(f"\n{'PASSED ✓' if result_6 else 'FAILED ✗'}")

---
## TEST 7: Forward Pass (Inference)
Tests that model can process sample data

In [ ]:
def test_forward_pass():
    """TEST 7: Test model forward pass with sample data"""
    print("\n" + "="*60)
    print("TEST 7: Forward Pass (Inference) Test")
    print("="*60)
    
    if model is None or X_test is None:
        print("✗ Model or data not loaded")
        return False
    
    try:
        # Test single sample
        sample = X_test[0:1]  # Shape: (1, 64, 64, 3)
        print(f"✓ Testing with single sample")
        print(f"  Input shape: {sample.shape}")
        
        predictions = model.predict(sample, verbose=0)
        print(f"\n✓ Forward pass successful")
        print(f"  Output shape: {predictions.shape}")
        print(f"  Probabilities: {predictions[0]}")
        print(f"  Predicted class: {np.argmax(predictions[0])}")
        
        # Test batch
        batch = X_test[0:32]
        print(f"\n✓ Testing with batch of 32 samples")
        print(f"  Input shape: {batch.shape}")
        
        predictions_batch = model.predict(batch, verbose=0)
        print(f"\n✓ Batch prediction successful")
        print(f"  Output shape: {predictions_batch.shape}")
        print(f"  Prediction range: [{predictions_batch.min():.4f}, {predictions_batch.max():.4f}]")
        
        # Check probability sums to 1
        sums = predictions_batch.sum(axis=1)
        if np.allclose(sums, 1.0):
            print(f"✓ Probabilities correctly sum to 1.0")
        else:
            print(f"⚠ Probability sums: min={sums.min():.6f}, max={sums.max():.6f}")
        
        return True
    
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

result_7 = test_forward_pass()
print(f"\n{'PASSED ✓' if result_7 else 'FAILED ✗'}")

---
## TEST 8: Mini Training (Overfitting Test)
Tests that model can train on a small subset (should overfit)

In [ ]:
def test_mini_training():
    """TEST 8: Test training on small subset (should overfit quickly)"""
    print("\n" + "="*60)
    print("TEST 8: Mini Training Test (Overfitting Expected)")
    print("="*60)
    
    if model is None or X_train is None:
        print("✗ Model or data not loaded")
        return False
    
    try:
        # Use only 256 samples for quick test
        X_mini = X_train[0:256]
        y_mini = y_train[0:256]
        
        print(f"✓ Training on mini dataset: {len(X_mini)} samples")
        print(f"  Class distribution: {np.bincount(y_mini)}")
        
        # Create fresh model
        model_mini = create_enhanced_cnn(input_shape=(PATCH_SIZE, PATCH_SIZE, 3))
        model_mini.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train for 3 epochs
        print(f"\n--- Training Progress ---")
        history = model_mini.fit(
            X_mini, y_mini,
            epochs=3,
            batch_size=32,
            verbose=1
        )
        
        print(f"\n✓ Training completed")
        
        # Check if loss decreased (overfitting)
        loss_prog = history.history['loss']
        print(f"\n--- Loss Progression ---")
        for epoch, loss in enumerate(loss_prog, 1):
            print(f"  Epoch {epoch}: {loss:.4f}")
        
        if loss_prog[-1] < loss_prog[0]:
            print(f"\n✓ Loss decreased (model is learning)")
        else:
            print(f"⚠ Loss did not decrease significantly")
        
        return True
    
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

result_8 = test_mini_training()
print(f"\n{'PASSED ✓' if result_8 else 'FAILED ✗'}")

---
## TEST 9: Metrics Computation
Tests evaluation metrics on sample predictions

In [ ]:
def test_metrics_computation():
    """TEST 9: Test metric computation"""
    print("\n" + "="*60)
    print("TEST 9: Metrics Computation Test")
    print("="*60)
    
    if model is None or X_test is None:
        print("✗ Model or data not loaded")
        return False
    
    try:
        # Get predictions on test set
        print(f"✓ Computing predictions on test set ({len(X_test)} samples)")
        y_pred_proba = model.predict(X_test, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)
        
        print(f"✓ Predictions computed")
        print(f"  Predicted class distribution: {np.bincount(y_pred)}")
        print(f"  True class distribution: {np.bincount(y_test)}")
        
        # Compute metrics
        print(f"\n--- Classification Metrics ---")
        
        accuracy = accuracy_score(y_test, y_pred)
        print(f"  Accuracy: {accuracy:.4f}")
        
        precision = precision_score(y_test, y_pred, zero_division=0)
        print(f"  Precision: {precision:.4f}")
        
        recall = recall_score(y_test, y_pred, zero_division=0)
        print(f"  Recall: {recall:.4f}")
        
        f1 = f1_score(y_test, y_pred, zero_division=0)
        print(f"  F1 Score: {f1:.4f}")
        
        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print(f"\n--- Confusion Matrix ---")
        print(f"  True Negatives (TN): {cm[0,0]}")
        print(f"  False Positives (FP): {cm[0,1]}")
        print(f"  False Negatives (FN): {cm[1,0]}")
        print(f"  True Positives (TP): {cm[1,1]}")
        
        # Classification report
        print(f"\n--- Detailed Classification Report ---")
        print(classification_report(y_test, y_pred, target_names=['No-Burn', 'Burn']))
        
        return True
    
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

result_9 = test_metrics_computation()
print(f"\n{'PASSED ✓' if result_9 else 'FAILED ✗'}")

---
## TEST 10: Visualization
Tests plotting and visualization functions

In [ ]:
def test_visualizations():
    """TEST 10: Test visualization functions"""
    print("\n" + "="*60)
    print("TEST 10: Visualization Test")
    print("="*60)
    
    if X_test is None or y_test is None:
        print("✗ Data not loaded")
        return False
    
    try:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        fig.suptitle('Wildfire Prediction - Test Visualizations', fontsize=14, fontweight='bold')
        
        # 1. Sample patches
        print(f"✓ Plotting sample patches...")
        for i in range(2):
            # Display patch as RGB (normalize to 0-1)
            patch = X_test[i]
            patch_rgb = np.clip(patch * 3, 0, 1)  # Scale for visibility
            axes[0, i].imshow(patch_rgb)
            axes[0, i].set_title(f'Patch {i} (Label: {y_test[i]})')
            axes[0, i].axis('off')
        
        # 2. Class distribution
        print(f"✓ Plotting class distribution...")
        classes = ['No-Burn', 'Burn']
        class_counts = np.bincount(y_test)
        axes[1, 0].bar(classes, class_counts, color=['green', 'red'])
        axes[1, 0].set_title('Test Set Class Distribution')
        axes[1, 0].set_ylabel('Count')
        for j, v in enumerate(class_counts):
            axes[1, 0].text(j, v, str(v), ha='center', va='bottom')
        
        # 3. Probability distribution
        print(f"✓ Plotting prediction probabilities...")
        if model is not None:
            y_pred_proba = model.predict(X_test[:1000], verbose=0)  # First 1000 for speed
            axes[1, 1].hist(y_pred_proba[:, 1], bins=30, edgecolor='black', alpha=0.7)
            axes[1, 1].set_title('Predicted Probability Distribution (Class 1)')
            axes[1, 1].set_xlabel('Probability')
            axes[1, 1].set_ylabel('Frequency')
        
        plt.tight_layout()
        plt.savefig(os.path.join(PROJECT_ROOT, 'test_visualizations.png'), dpi=100, bbox_inches='tight')
        print(f"\n✓ Visualizations saved to 'test_visualizations.png'")
        plt.show()
        
        return True
    
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

result_10 = test_visualizations()
print(f"\n{'PASSED ✓' if result_10 else 'FAILED ✗'}")

---
## TEST SUMMARY
Summary of all test results

In [ ]:
# Summary of all tests
print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)

tests = [
    ("TEST 1", "Data Files Integrity", result_1),
    ("TEST 2", "CSV Metadata Validation", result_2),
    ("TEST 3", "GeoTIFF Integrity Check", result_3),
    ("TEST 4", "Patch Extraction & Normalization", result_4),
    ("TEST 5", "Train-Test Split Validation", result_5),
    ("TEST 6", "Model Architecture Validation", result_6),
    ("TEST 7", "Forward Pass (Inference) Test", result_7),
    ("TEST 8", "Mini Training Test", result_8),
    ("TEST 9", "Metrics Computation Test", result_9),
    ("TEST 10", "Visualization Test", result_10),
]

passed = 0
failed = 0

print(f"\n{'Test':<10} {'Description':<40} {'Result':<15}")
print("-" * 70)

for test_id, description, result in tests:
    status = "✓ PASSED" if result else "✗ FAILED"
    if result:
        passed += 1
    else:
        failed += 1
    print(f"{test_id:<10} {description:<40} {status:<15}")

print("-" * 70)
print(f"\nTotal: {passed} PASSED, {failed} FAILED out of 10 tests")
print(f"Success Rate: {(passed/10)*100:.0f}%")

if passed == 10:
    print("\n🎉 ALL TESTS PASSED! MVP pipeline is working.")
elif passed >= 7:
    print("\n✓ Most tests passed. Minor issues to address.")
else:
    print("\n⚠ Multiple test failures. Review errors above.")

---
## Notes for GSOC Demo

**What this test suite validates:**

1. ✓ **Data Pipeline**: GEE export → CSV metadata → TIF loading
2. ✓ **Data Quality**: Normalization, NaN handling, class distribution
3. ✓ **Model Architecture**: CNN with residual blocks compiles correctly
4. ✓ **Training Loop**: Forward pass, loss computation, gradient updates
5. ✓ **Evaluation**: Metrics (Accuracy, F1, Recall, Precision)
6. ✓ **Reproducibility**: Stratified splits, random seeds, version control

**Next Steps:**
- [ ] Run full training (20 epochs) on Colab with T4 GPU
- [ ] Implement Focal Loss for class imbalance
- [ ] Add SMOTE oversampling
- [ ] Compare with Random Forest baseline
- [ ] Create PR to original repository

**Known Issues to Address:**
- F1 score = 0 (class imbalance → implement weighted loss)
- Model predicts all samples as negative class (threshold tuning needed)
- Some NaN patches detected (filtering is working correctly)
